In [10]:
    import requests
    import pandas as pd
    from datetime import date
    import numpy as np

    # URL template (corrected)
    url = "https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15"

    # Instrument tokens
    tickerData = {"NIFTY": {"id": 256265}, "BANKNIFTY": {"id": 260105}}

    # Interval mapping
    KI = {
        "3m": "3minute",
        "5m": "5minute",
        "15m": "15minute",
        "60m": "60minute",
        "day": "day",
    }

    # Function to get candle data
    def getCandles(symbol, fromDate, toDate, timeframe):
        headers = {
            "authorization": "enctoken l9RBs2mUaCOtHh76/Al766xp7yJ1ndunq9zE2PslZvpMEXu0ccRXlWhWHNUOyJX19msbiNnAcd37PdO0yb1ikRiT3q1N7v2hutAK2FB3s2GAGxkfVgusyQ=="
            # Replace this with your actual token securely
        }
        curUrl = url.format(tickerData[symbol]["id"], timeframe, fromDate, toDate)
        print("Requesting:", curUrl)

        try:
            session = requests.session()
            response = session.get(curUrl, headers=headers)
            r = response.json()
        except Exception as error:
            print("API request failed:", error)
            return pd.DataFrame()

        try:
            history = r["data"]["candles"]
            history_df = pd.DataFrame(history)
            history_df = history_df.rename(
                columns={
                    0: "Datetime",
                    1: "open",
                    2: "high",
                    3: "low",
                    4: "close",
                    5: "volume",
                }
            )
            history_df["Datetime"] = pd.to_datetime(history_df["Datetime"])
            history_df.set_index("Datetime", inplace=True)
            return history_df
        except Exception as error:
            print("Data processing failed:", error)
            return pd.DataFrame()


    def ATR(df, n):
        df = df.copy()
        df["H-L"] = abs(df["high"] - df["low"])
        df["H-PC"] = abs(df["high"] - df["close"].shift(1))
        df["L-PC"] = abs(df["low"] - df["close"].shift(1))
        df["TR"] = df[["H-L", "H-PC", "L-PC"]].max(axis=1, skipna=False)
        atr = df["TR"].ewm(span=n, adjust=False, min_periods=n).mean()
        return atr  

    # Supertrend implementation
    def supertrend(DF, n=7, m=3):
        """
        Function to calculate Supertrend.
        n : ATR period (default 7)
        m : Multiplier (default 3)
        """
        df = DF.copy()
        df["ATR"] = ATR(df, n)["ATR"]


        hl2 = (df["high"] + df["low"]) / 2
        df["UpperBand"] = hl2 + m * df["ATR"]
        df["LowerBand"] = hl2 - m * df["ATR"]
        df["Supertrend"] = np.nan
        df["in_uptrend"] = True  # boolean flag for trend

        for current in range(n, len(df)):
            previous = current - 1

            if df["close"][current] > df["UpperBand"][previous]:
                df.at[current, "in_uptrend"] = True
            elif df["close"][current] < df["LowerBand"][previous]:
                df.at[current, "in_uptrend"] = False
            else:
                df.at[current, "in_uptrend"] = df.at[previous, "in_uptrend"]
                if (
                    df["in_uptrend"][current]
                    and df["LowerBand"][current] < df["LowerBand"][previous]
                ):
                    df.at[current, "LowerBand"] = df["LowerBand"][previous]
                if (
                    not df["in_uptrend"][current]
                    and df["UpperBand"][current] > df["UpperBand"][previous]
                ):
                    df.at[current, "UpperBand"] = df["UpperBand"][previous]

            # Set final supertrend value
            if df["in_uptrend"][current]:
                df.at[current, "Supertrend"] = df["LowerBand"][current]
            else:
                df.at[current, "Supertrend"] = df["UpperBand"][current]

        return df[["close", "Supertrend", "in_uptrend"]]


    print("Start")
    symbol_data = getCandles("NIFTY", "2025-03-20", "2025-03-25", KI["15m"])


    if not symbol_data.empty:
        result = supertrend(symbol_data, n=7, m=3)
        print(symbol_data.columns)
    else:
        print("No OHLC data available.")


Start
Requesting: https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15


KeyError: 'ATR'